# Basketabll Activity Recognition using LSTM Neural Network




# Reading the dataset

In [1]:
import pandas as pd
import numpy as np
import glob

import tensorflow as tf
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader

from sklearn.metrics import precision_score, recall_score, f1_score, jaccard_score,confusion_matrix, plot_confusion_matrix
import time
from typing import Any
import torch
import torch.nn as nn

import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, LSTM
from sklearn.model_selection import KFold, StratifiedKFold, RepeatedKFold
from sklearn.metrics import f1_score, confusion_matrix, plot_confusion_matrix
from pandas import DataFrame
from keras.models import load_model

files = glob.glob('/data/shk/dl-for-har/tutorial_notebooks/Prof_New/*.csv')

name = [file.split('/')[-1] for file in files]
df = pd.concat(map(pd.read_csv,files),ignore_index = True) #4230213 rows × 10 columns

data = df

#Considering only basketball
clean_data = data[data['basketball']!='not_labeled']
clean_data = clean_data[data['basketball'].notna()]
clean_data['basketball'].replace({'jumping':'layup'},inplace=True) #Replacing jumping by layup

new_data = clean_data[clean_data.coarse != 'game'] #623758 rows × 10 columns
new_data = new_data.drop(columns='coarse')
new_data = new_data.iloc[:,:6]
print('new_data',new_data.shape)
print('____')

# Sepearting Game data from train data
game_data = clean_data[clean_data.coarse == 'game']
game_data = game_data.drop(columns='coarse')
game_data = game_data.iloc[:,:6] #45446 rows × 6 columns
print('game_data',game_data.shape)

2022-08-14 20:31:38.961504: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-14 20:31:38.961528: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/shk/anaconda3/envs/adversarial/lib/python3.7/site-packages/ipykernel_launcher.py:33: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


new_data (613454, 6)
____
game_data (45111, 6)


# Labelling the data

In [2]:
label = new_data['basketball']
x_axis = sorted(label.unique())
y_axis = label.value_counts()

X = new_data.iloc[:,:-1]#.astype(np.float32)

class_names = ['dribbling','shot','pass','layup','rebound']
num_classes = len(class_names)

def labelling(clean_data,data_y):
    clean_data[data_y == 'dribbling'] = 0
    clean_data[data_y == 'shot'] = 1
    clean_data[data_y == 'pass'] = 2
    clean_data[data_y == 'layup'] = 3
    clean_data[data_y == 'rebound'] = 4
    
    return data_y

y = labelling(new_data,new_data.iloc[:,-1]).astype(int)

#subject = X['subject']
#uni_subject = subject.unique()
#uni_subject_count = subject.value_counts()

data_labelled = np.concatenate((X, y[:,None]), axis=1)[:,1:] #669204, 6


/home/shk/anaconda3/envs/adversarial/lib/python3.7/site-packages/ipykernel_launcher.py:25: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.


# Special preprocessing

In [3]:
data_bribbling = data_labelled[data_labelled[:,-1] == 0]
data_shot = data_labelled[data_labelled[:,-1] == 1]
data_pass = data_labelled[data_labelled[:,-1] == 2]
data_layup = data_labelled[data_labelled[:,-1] == 3]
data_rebound = data_labelled[data_labelled[:,-1] == 4]


def spilt(data):

    dd = np.asarray(data)
    T = int(0.8* len(dd))
    train_size = int(T)
    test_size = len(dd) - train_size

    train_df,test_df = torch.utils.data.random_split(dd, [train_size,test_size])
    X = train_df[:][:,0:4]
    Y = train_df[:][:,4]

    X_v = test_df[:][:,0:4]
    y_v = test_df[:][:,4]
  
    return X,Y,X_v,y_v

X_dribble,Y_dribble,X_v_dribble,y_v_dribble = spilt(data_bribbling)
X_shot,Y_shot,X_v_shot,y_v_shot = spilt(data_shot)
X_pass,Y_pass,X_v_pass,y_v_pass = spilt(data_pass)
X_layup,Y_layup,X_v_layup,y_v_layup = spilt(data_layup)
X_rebound,Y_rebound,X_v_rebound,y_v_rebound = spilt(data_rebound)

X = np.concatenate((X_dribble, X_shot,X_pass,X_layup,X_rebound), axis=0)[:,1:]
Y = np.concatenate((Y_dribble, Y_shot,Y_pass,Y_layup,Y_rebound), axis=0)

X_v = np.concatenate((X_v_dribble, X_v_shot,X_v_pass,X_v_layup,X_v_rebound), axis=0)[:,1:]
y_v = np.concatenate((y_v_dribble, y_v_shot,y_v_pass,y_v_layup,y_v_rebound), axis=0)

train_data = np.column_stack((X,Y))
valid_data = np.column_stack((X_v,y_v))

# Sliding Window

In [4]:
def sliding_window(data, samples_per_window, overlap_ratio):
    windows = []
    indices = []
    curr = 0
    win_len = int(samples_per_window)
    
    if overlap_ratio is not None:
        overlapping_elements = int((overlap_ratio / 100) * (win_len))
        if overlapping_elements >= win_len:
            print('Number of overlapping elements exceeds window size.')
            return
    while curr < len(data) - win_len:
        windows.append(data[curr:curr + win_len])
        indices.append([curr, curr + win_len])
        curr = curr + win_len - overlapping_elements
    try:
        result_windows = np.array(windows)
        result_indices = np.array(indices)
    except:
        result_windows = np.empty(shape=(len(windows), win_len, data.shape[1]), dtype=object)
        result_indices = np.array(indices)
        for i in range(0, len(windows)):
            result_windows[i] = windows[i]
            result_indices[i] = indices[i]
    return result_windows, result_indices

def apply_sliding_window(data_x, data_y, sliding_window_size, sampling_rate, sliding_window_overlap):
    
    output_x, _ = sliding_window(data_x, sliding_window_size, sliding_window_overlap)
    output_y, _ = sliding_window(data_y, sliding_window_size, sliding_window_overlap)

    return output_x,output_y

sw_length = 50
sw_overlap = 25

X_train, y_train = apply_sliding_window(X, Y, sw_length, sampling_rate=50,sliding_window_overlap=sw_overlap)
X_valid, y_valid = apply_sliding_window(X_v, y_v, sw_length, sampling_rate=50,sliding_window_overlap=sw_overlap)


X_train, y_train = X_train.astype(np.float32), y_train.astype(np.uint8)
X_valid, y_valid = X_valid.astype(np.float32), y_valid.astype(np.uint8)

print("\nShape of the X_train and Y_train datasets after windowing: ")
print(X_train.shape, y_train.shape) 

print("\nShape of the X_valid and y_valid datasets after windowing: ")
print(X_valid.shape, y_valid.shape)

y_train = y_train[:,0]
y_valid = y_valid[:,0]

data_X = np.concatenate((X_train,X_valid),axis=0) 

print("\nShape of the X after windowing: ")
print(data_X.shape)

print("\nShape of the Y after windowing: ")
data_Y = np.concatenate((y_train,y_valid),axis=0) 
print(data_Y.shape)


Shape of the X_train and Y_train datasets after windowing: 
(12914, 50, 3) (12914, 50)

Shape of the X_valid and y_valid datasets after windowing: 
(3228, 50, 3) (3228, 50)

Shape of the X after windowing: 
(16142, 50, 3)

Shape of the Y after windowing: 
(16142,)


# Network

In [5]:
def Network(X_train,y_train,X_valid, y_valid):

    train_preds = []
    train_gt = []
    test_preds = []
    test_gt = []   
    loss_acc = []
    c_matrix =[]

    model = Sequential()
    model.add(LSTM(units=26,input_shape=[X_train.shape[1], X_train.shape[2]], activation = 'relu', return_sequences = True))
    model.add(Dropout(rate=0.1))
    model.add(Flatten())
    model.add(Dense(units=1300, activation='relu'))
    model.add(Dropout(rate=0.2))

    model.add(Dense(units=850, activation='relu'))
    model.add(Dropout(rate=0.3))
    model.add(Dense(units=5, activation='softmax'))

    model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), 
                  metrics=['accuracy'])

    model.fit(X_train,y_train,epochs=20, validation_data=(X_valid,y_valid),verbose=1)

    y_pred_train = np.argmax(model.predict(X_train), axis=-1)
    y_pred_test = np.argmax(model.predict(X_valid), axis=-1)
        
        
    #Training
    acc_train = jaccard_score(y_train, y_pred_train, average='macro')
    pre_train = precision_score(y_train, y_pred_train, average='macro', zero_division=0)
    reca_train = recall_score(y_train, y_pred_train, average='macro', zero_division=0)
    f1_train = f1_score(y_train, y_pred_train, average='macro', zero_division=0)

    train_preds = np.concatenate((np.array(train_preds, int), np.array(y_pred_train, int)))
    train_gt = np.concatenate((np.array(train_gt, int), np.array(y_train, int)))

    print("___Training____")
    print(f"accuracy {round(acc_train,3)*100} ,precision {round(pre_train,3)*100}, recall {round(reca_train,3)*100},f1_score {round(f1_train,3)*100} ")

        #Validation
    acc_test = jaccard_score(y_valid, y_pred_test, average='macro')
    pre_test = precision_score(y_valid, y_pred_test, average='macro', zero_division=0)
    reca_test = recall_score(y_valid, y_pred_test, average='macro', zero_division=0)
    f1_test = f1_score(y_valid, y_pred_test, average='macro', zero_division=0)

    test_preds = np.concatenate((np.array(test_preds, int), np.array(y_pred_test, int)))
    test_gt = np.concatenate((np.array(test_gt, int), np.array(y_valid, int)))

    print("___Validation____")
    print(f"accuracy {round(acc_test,3)*100} ,precision {round(pre_test,3)*100}, recall {round(reca_test,3)*100}, f1_score {round(f1_test,3)*100} ")

        #Genarate Metrics
    loss_acc.append(model.evaluate(X_valid,y_valid))
    c_matrix.append(confusion_matrix(y_valid, y_pred_test))   

        #Save Weights
    model.save_weights('./Pre_Trained_models/without_k_fold/without_k_fold',overwrite=True)
    model.save('./Pre_Trained_models/without_k_fold',overwrite=True)
    
    return train_preds,train_gt,test_preds,test_gt,loss_acc,matrix

In [6]:
train_preds,train_gt,test_preds,test_gt,loss_acc,c_matrix = Network(X_train,y_train,X_valid, y_valid)

print(train_preds.shape)
print(train_gt.shape)
print()

print(test_preds.shape)
print(test_gt.shape)
#print(scores)

val_loss,val_acc = loss_acc[-1]
print(f'validataion_loss {round(val_loss,3)}, validation_accuracy {round(val_acc,3)*100}')

Epoch 1/20


2022-08-14 20:31:46.451809: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_COMPAT_NOT_SUPPORTED_ON_DEVICE: forward compatibility was attempted on non supported HW
2022-08-14 20:31:46.451832: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: DeepCube
2022-08-14 20:31:46.451837: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: DeepCube
2022-08-14 20:31:46.451898: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: NOT_FOUND: was unable to find libcuda.so DSO loaded into this program
2022-08-14 20:31:46.451919: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 470.129.6
2022-08-14 20:31:46.452137: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operat

404/404 [==============================] - 8s 19ms/step - loss: 0.4884 - accuracy: 0.8413 - val_loss: 0.3323 - val_accuracy: 0.8903
Epoch 2/20
404/404 [==============================] - 8s 19ms/step - loss: 0.2969 - accuracy: 0.8973 - val_loss: 0.3421 - val_accuracy: 0.8727
Epoch 3/20
404/404 [==============================] - 8s 19ms/step - loss: 0.2371 - accuracy: 0.9164 - val_loss: 0.2160 - val_accuracy: 0.9275
Epoch 4/20
404/404 [==============================] - 8s 19ms/step - loss: 0.2059 - accuracy: 0.9291 - val_loss: 0.2229 - val_accuracy: 0.9229
Epoch 5/20
404/404 [==============================] - 8s 19ms/step - loss: 0.1776 - accuracy: 0.9379 - val_loss: 0.2491 - val_accuracy: 0.9043
Epoch 6/20
404/404 [==============================] - 8s 19ms/step - loss: 0.1599 - accuracy: 0.9408 - val_loss: 0.1887 - val_accuracy: 0.9368
Epoch 7/20
404/404 [==============================] - 7s 19ms/step - loss: 0.1461 - accuracy: 0.9480 - val_loss: 0.1899 - val_accuracy: 0.9346
Epoch 8/20

2022-08-14 20:34:20.762330: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ./without_k_fold/assets


(12914,)
(12914,)

(3228,)
(3228,)
validataion_loss 0.241, validation_accuracy 93.89999999999999


# Predictions

In [7]:
cls = np.array(range(5))
class_names = ['dribbling','shot','pass','layup','rebound']

print('\nTraining RESULTS: ')
print("\nAvg. Accuracy: {0}".format(jaccard_score(train_gt, train_preds, average='macro')))
print("Avg. Precision: {0}".format(precision_score(train_gt, train_preds, average='macro')))
print("Avg. Recall: {0}".format(recall_score(train_gt, train_preds, average='macro')))
print("Avg. F1: {0}".format(f1_score(train_gt, train_preds, average='macro')))

print("\nTraining RESULTS (PER CLASS): ")
print("\nAccuracy:")
for i, rslt in enumerate(jaccard_score(train_gt, train_preds, average=None, labels=cls)):
    print("   {0}: {1} %".format(class_names[i], rslt*100))
print("\nPrecision:")
for i, rslt in enumerate(precision_score(train_gt, train_preds, average=None, labels=cls)):
    print("   {0}: {1} %".format(class_names[i], rslt*100))
print("\nRecall:")
for i, rslt in enumerate(recall_score(train_gt, train_preds, average=None, labels=cls)):
    print("   {0}: {1} %".format(class_names[i], rslt*100))
print("\nF1:")
for i, rslt in enumerate(f1_score(train_gt, train_preds, average=None, labels=cls)):
    print("   {0}: {1} %".format(class_names[i], rslt*100))


Training RESULTS: 

Avg. Accuracy: 0.9763667675887608
Avg. Precision: 0.9915311667513851
Avg. Recall: 0.9846684481394906
Avg. F1: 0.9879248469592117

Training RESULTS (PER CLASS): 

Accuracy:
   dribbling: 99.49322577308925 %
   shot: 98.69727047146401 %
   pass: 95.03042596348884 %
   layup: 99.84051036682615 %
   rebound: 95.1219512195122 %

Precision:
   dribbling: 99.76148501503681 %
   shot: 100.0 %
   pass: 96.00409836065575 %
   layup: 100.0 %
   rebound: 100.0 %

Recall:
   dribbling: 99.73045822102425 %
   shot: 98.69727047146401 %
   pass: 98.9440337909187 %
   layup: 99.84051036682615 %
   rebound: 95.1219512195122 %

F1:
   dribbling: 99.74596920524651 %
   shot: 99.344364658133 %
   pass: 97.45189807592305 %
   layup: 99.92019154030328 %
   rebound: 97.5 %


# Validation Accuracies

In [8]:
print('\nValidation RESULTS: ')
print("\nAvg. Accuracy: {0}".format(jaccard_score(test_gt, test_preds, average='macro')))
print("Avg. Precision: {0}".format(precision_score(test_gt, test_preds, average='macro')))
print("Avg. Recall: {0}".format(recall_score(test_gt, test_preds, average='macro')))
print("Avg. F1: {0}".format(f1_score(test_gt, test_preds, average='macro')))

print("\nVALIDATION RESULTS (PER CLASS): ")
print("\nAccuracy:")
for i, rslt in enumerate(jaccard_score(test_gt, test_preds, average=None, labels=cls)):
    print(f'{class_names[i]}: {round(rslt,2)*100}')
    #print("   {0}: {1} %".format(class_names[i], rslt*100))
print("\nPrecision:")
for i, rslt in enumerate(precision_score(test_gt, test_preds, average=None, labels=cls)):
    print(f'{class_names[i]}: {round(rslt,2)*100}')
    #print("   {0}: {1} %".format(class_names[i], rslt*100))
print("\nRecall:")
for i, rslt in enumerate(recall_score(test_gt, test_preds, average=None, labels=cls)):
    print(f'{class_names[i]}: {round(rslt,2)*100}')
    #print("   {0}: {1} %".format(class_names[i], rslt*100))
print("\nF1:")
for i, rslt in enumerate(f1_score(test_gt, test_preds, average=None, labels=cls)):
    print(f'{class_names[i]}: {round(rslt,2)*100}')
    #print("   {0}: {1} %".format(class_names[i], rslt*100))
    
print("\nGENERALIZATION GAP ANALYSIS: ")
print("\nTrain-Val-Accuracy Difference: {0}".format(jaccard_score(train_gt, train_preds, average='macro') -
                                                  jaccard_score(test_gt, test_preds, average='macro')))
print("Train-Val-Precision Difference: {0}".format(precision_score(train_gt, train_preds, average='macro') -
                                                   precision_score(test_gt, test_preds, average='macro')))
print("Train-Val-Recall Difference: {0}".format(recall_score(train_gt, train_preds, average='macro') -
                                                recall_score(test_gt, test_preds, average='macro')))
print("Train-Val-F1 Difference: {0}".format(f1_score(train_gt, train_preds, average='macro') -
                                            f1_score(test_gt, test_preds, average='macro')))


Validation RESULTS: 

Avg. Accuracy: 0.7513672746959177
Avg. Precision: 0.9082467170018422
Avg. Recall: 0.8046479374546267
Avg. F1: 0.8464381806604953

VALIDATION RESULTS (PER CLASS): 

Accuracy:
dribbling: 93.0
shot: 85.0
pass: 56.00000000000001
layup: 89.0
rebound: 52.0

Precision:
dribbling: 95.0
shot: 97.0
pass: 72.0
layup: 98.0
rebound: 92.0

Recall:
dribbling: 98.0
shot: 87.0
pass: 72.0
layup: 91.0
rebound: 55.00000000000001

F1:
dribbling: 96.0
shot: 92.0
pass: 72.0
layup: 94.0
rebound: 69.0

GENERALIZATION GAP ANALYSIS: 

Train-Val-Accuracy Difference: 0.22499949289284316
Train-Val-Precision Difference: 0.08328444974954285
Train-Val-Recall Difference: 0.1800205106848639
Train-Val-F1 Difference: 0.14148666629871642


# Testing on Game data

In [9]:
print('game_data',game_data.shape)

label_game = game_data['basketball']
x_axis_game = sorted(label_game.unique())
y_axis_game = label_game.value_counts()

X_game = game_data.iloc[:,:-1]

def labelling(clean_data,data_y):
    clean_data[data_y == 'dribbling'] = 0
    clean_data[data_y == 'shot'] = 1
    clean_data[data_y == 'pass'] = 2
    clean_data[data_y == 'layup'] = 3
    clean_data[data_y == 'rebound'] = 4
    
    return data_y

y_game = labelling(game_data,game_data.iloc[:,-1]).astype(int)

data_labelled_game = np.concatenate((X_game, y_game[:,None]), axis=1)[:,2:] #669204, 6
print('data_labelled_game ',data_labelled_game.shape)

data_labelled_X = data_labelled_game[:,:-1]
data_labelled_Y = data_labelled_game[:,-1]

print(data_labelled_X.shape,data_labelled_Y.shape)
sw_length = 50
sw_overlap = 25

X_train_game, y_train_game = apply_sliding_window(data_labelled_X, data_labelled_Y, sliding_window_size=sw_length, 
                                        sampling_rate=50,sliding_window_overlap=sw_overlap)


X_train_game, y_train_game = X_train_game.astype(np.float32), y_train_game[:,0].astype(np.uint8)

print("\nShape of the X_train_game and y_train_game datasets after windowing: ")
print(X_train_game.shape, y_train_game.shape)

game_data (45111, 6)
data_labelled_game  (45111, 4)
(45111, 3) (45111,)

Shape of the X_train_game and y_train_game datasets after windowing: 
(1186, 50, 3) (1186,)


/home/shk/anaconda3/envs/adversarial/lib/python3.7/site-packages/ipykernel_launcher.py:20: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.


# Loading the pretrained model

In [10]:
model = load_model('./Pre_Trained_models/without_k_fold')

pt = './Pre_Trained_models/without_k_fold/without_k_fold'
model.load_weights(pt)

In [12]:
X_tt = data_X
y_tt = data_Y
loss_drill,acc_drill = model.evaluate(X_tt,y_tt)

print('___DRILL___')
print(f'DRILL_loss {round(loss_drill,3)}, Drill_accuracy {round(acc_drill,3)*100}')

#X_tt = data_X
#y_tt = data_Y
loss_game,acc_game = model.evaluate(X_train_game,y_train_game)

print('___GAME___')
print(f'GAME_loss {round(loss_game,3)}, GAME_accuracy {round(acc_game,3)*100}')

505/505 [==============================] - 2s 4ms/step - loss: 0.0631 - accuracy: 0.9839
___DRILL___
DRILL_loss 0.063, Drill_accuracy 98.4
38/38 [==============================] - 0s 4ms/step - loss: 2023.2369 - accuracy: 0.3010
___GAME___
GAME_loss 2023.237, GAME_accuracy 30.099999999999998


In [13]:

y_pred_game = np.argmax(model.predict(X_train_game), axis=-1)

#Training
acc_game = jaccard_score(y_train_game, y_pred_game, average='macro')
pre_game = precision_score(y_train_game, y_pred_game, average='macro', zero_division=0)
reca_game = recall_score(y_train_game, y_pred_game, average='macro', zero_division=0)
f1_game = f1_score(y_train_game, y_pred_game, average='macro', zero_division=0)

print("___GAME____")
print(f"accuracy {round(acc_game,3)*100} ,precision {round(pre_game,3)*100}, recall {round(reca_game,3)*100},f1_score {round(f1_game,3)*100} ")

___GAME____
accuracy 15.4 ,precision 26.1, recall 29.4,f1_score 25.900000000000002 


In [14]:
print('\nGame RESULTS: ')
print("\nAvg. Accuracy: {0}".format(jaccard_score(y_train_game, y_pred_game, average='macro')))
print("Avg. Precision: {0}".format(precision_score(y_train_game, y_pred_game, average='macro')))
print("Avg. Recall: {0}".format(recall_score(y_train_game, y_pred_game, average='macro')))
print("Avg. F1: {0}".format(f1_score(y_train_game, y_pred_game, average='macro')))

print("\nGAME RESULTS (PER CLASS): ")
print("\nAccuracy:")
for i, rslt in enumerate(jaccard_score(y_train_game, y_pred_game, average=None, labels=cls)):
    print(f'{class_names[i]}: {round(rslt,2)*100}')
    #print("   {0}: {1} %".format(class_names[i], rslt*100))
print("\nPrecision:")
for i, rslt in enumerate(precision_score(y_train_game, y_pred_game, average=None, labels=cls)):
    print(f'{class_names[i]}: {round(rslt,2)*100}')
    #print("   {0}: {1} %".format(class_names[i], rslt*100))
print("\nRecall:")
for i, rslt in enumerate(recall_score(y_train_game, y_pred_game, average=None, labels=cls)):
    print(f'{class_names[i]}: {round(rslt,2)*100}')
    #print("   {0}: {1} %".format(class_names[i], rslt*100))
print("\nF1:")
for i, rslt in enumerate(f1_score(y_train_game, y_pred_game, average=None, labels=cls)):
    print(f'{class_names[i]}: {round(rslt,2)*100}')
    #print("   {0}: {1} %".format(class_names[i], rslt*100))


Game RESULTS: 

Avg. Accuracy: 0.1538711331959549
Avg. Precision: 0.2610529254281275
Avg. Recall: 0.29410973562860565
Avg. F1: 0.25934556345335896

GAME RESULTS (PER CLASS): 

Accuracy:
dribbling: 27.0
shot: 10.0
pass: 13.0
layup: 21.0
rebound: 6.0

Precision:
dribbling: 51.0
shot: 14.000000000000002
pass: 27.0
layup: 26.0
rebound: 12.0

Recall:
dribbling: 37.0
shot: 31.0
pass: 19.0
layup: 50.0
rebound: 10.0

F1:
dribbling: 43.0
shot: 19.0
pass: 22.0
layup: 34.0
rebound: 11.0


# END